<a href="https://colab.research.google.com/github/FRNEYRA10/FRNEYRA10/blob/main/M%C3%A9todo_de_jacobi_ecu_de_Laplace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np

# Parametros
L = 2
h = 2/3
N = int(round(L/h))   # N = 3 subintervalos
nx = ny = N + 1       # 4 nodos por direccion: 0, 2/3, 4/3, 2
x = np.linspace(0, L, nx)
y = np.linspace(0, L, ny)

# Inicializar malla
u = np.zeros((nx, ny), dtype=float)

# Condiciones de frontera
# x=0 (izquierda)
u[0, :] = 0.0

# x=2 (derecha): u(2,y) = y(2-y)
for j, yj in enumerate(y):
    u[-1, j] = yj * (2.0 - yj)

# y=0 (abajo)
u[:, 0] = 0.0

# y=2 (arriba)
for i, xi in enumerate(x):
    if 0 < xi < 1:
        u[i, -1] = xi
    elif 1 <= xi < 2:
        u[i, -1] = 2.0 - xi
    else:
        u[i, -1] = 0.0

u_boundary = u.copy()  # guardamos las fronteras

# Metodo de Jacobi
tol = 1e-10
max_iter = 20000
u_old = u.copy()

for k in range(max_iter):
    u_new = u_old.copy()
    # actualizar  los puntos interiores
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            u_new[i, j] = 0.25 * (u_old[i+1, j] + u_old[i-1, j] + u_old[i, j+1] + u_old[i, j-1])

    diff = np.max(np.abs(u_new - u_old))
    u_old = u_new
    if diff < tol:
        break

u_jacobi = u_new.copy()
iterations = k + 1

# Solucion directa del sistema discreto (4x4)
A = np.array([[4, -1, -1, 0],
              [-1, 4, 0, -1],
              [-1, 0, 4, -1],
              [0, -1, -1, 4]], dtype=float)
b = np.array([0.0, 8.0/9.0, 2.0/3.0, 14.0/9.0], dtype=float)
sol = np.linalg.solve(A, b)

u_direct = u_boundary.copy()
u_direct[1,1] = sol[0]
u_direct[2,1] = sol[1]
u_direct[1,2] = sol[2]
u_direct[2,2] = sol[3]

# Mostrar resultados
np.set_printoptions(precision=10, suppress=True)
print("Nodos x:", x)
print("Nodos y:", y)
print("\nValores de frontera:\n", u_boundary)
print("\nResultado del metodo de Jacobi:")
print(f"Iteraciones: {iterations}, cambio maximo final: {diff:.3e}\n")
print(u_jacobi)
print("\nResultado por solucion directa del sistema discreto (4x4):\n")
print(u_direct)

interior_jacobi = np.array([u_jacobi[1,1], u_jacobi[2,1], u_jacobi[1,2], u_jacobi[2,2]])
print("\nValores interiores (orden: u11, u21, u12, u22):")
print("Jacobi:", interior_jacobi.tolist())
print("Directa:", sol.tolist())
print("Diferencia maxima |Jacobi - Directa| =", np.max(np.abs(interior_jacobi - sol)))

Nodos x: [0.           0.6666666667 1.3333333333 2.          ]
Nodos y: [0.           0.6666666667 1.3333333333 2.          ]

Valores de frontera:
 [[0.           0.           0.           0.          ]
 [0.           0.           0.           0.6666666667]
 [0.           0.           0.           0.6666666667]
 [0.           0.8888888889 0.8888888889 0.          ]]

Resultado del metodo de Jacobi:
Iteraciones: 32, cambio maximo final: 9.055e-11

[[0.           0.           0.           0.          ]
 [0.           0.1944444444 0.361111111  0.6666666667]
 [0.           0.4166666666 0.5833333332 0.6666666667]
 [0.           0.8888888889 0.8888888889 0.          ]]

Resultado por solucion directa del sistema discreto (4x4):

[[0.           0.           0.           0.          ]
 [0.           0.1944444444 0.3611111111 0.6666666667]
 [0.           0.4166666667 0.5833333333 0.6666666667]
 [0.           0.8888888889 0.8888888889 0.          ]]

Valores interiores (orden: u11, u21, u12, u2